In [1]:
import numpy as np
import pandas as pd

from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord

from pypika import Table, CustomFunction
from pypika.dialects import MSSQLQuery

from astroquery.gaia import Gaia

import matplotlib.pyplot as plt
from ipywidgets import interactive, Dropdown

In [2]:
def queryStars(coor, radius, dist_min, dist_max, top=100):
    src = Table("gaiadr3.gaia_source_lite")
    funcDistance = CustomFunction('DISTANCE', ['ra_value', 'dec_value', 'ra_label', 'dec_label'])

    q = MSSQLQuery \
        .from_(src) \
        .select(src.source_id, src.ra, src.dec, src.pmra, src.pmdec, src.parallax, src.phot_g_mean_mag, src.bp_rp, src.radial_velocity, src.teff_gspphot, src.logg_gspphot, src.mh_gspphot, src.distance_gspphot) \
        .top(top) \
        .where(funcDistance(coor.ra.value, coor.dec.value, 'ra', 'dec') < radius) \
        .where(src.distance_gspphot <= dist_max) \
        .where(src.distance_gspphot >= dist_min) \
        .where(src.ruwe <1.4) \
        .where(src.parallax_over_error >10)

    query_sql = q.get_sql() 
    query_sql = query_sql.replace('"', '')                       # Remove "" around any select
    query_sql = query_sql.replace("'ra'", "ra")                  # Remove "" around ra in DISTANCE function if used
    query_sql = query_sql.replace("'dec'", "dec")                # Remove "" around dec in DISTANCE function if used
    query_sql = query_sql.replace(f"TOP ({top})", f"TOP {top}")  # Remove "()" around TOP

    job     = Gaia.launch_job_async(query_sql)
    results = job.get_results()
    df = results.to_pandas()
    df['source_id'] = df['source_id'].astype(str)
    return df

In [3]:
#Pleiades (M45)
# Coordinates: RA = 03h 47m 00s, DEC = +24° 07' 00"
# Approximate distance: 136.2 pc (parallax = 7.364 mas)
# Distance range: 122.6 - 149.8 pc
# Suggested search radius: 2 degrees
# Expected Radial velocity: ~5.6 km/s (toward us)
coord_pleiades = SkyCoord('03h47m00s', '+24d07m00s', frame='icrs')
df_pleiades = queryStars(coord_pleiades, 2, 122.6, 149.8)


# Hyades
# Coordinates: RA = 04h 27m 00s, DEC = +15° 52' 00"
# Approximate distance: 47.0 pc (parallax = 21.278 mas)
# Distance range: 42.3 - 51.7 pc
# Suggested search radius: 5 degrees
coord_hyades = SkyCoord('04h27m00s', '+15d52m00s', frame='icrs')
df_hyades = queryStars(coord_hyades, 5, 42.3, 51.7)


# Beehive Cluster (M44)
# Coordinates: RA = 08h 40m 24s, DEC = +19° 40' 00"
# Approximate distance: 181 pc (parallax = 5.512 mas)
# Distance range: 162.9 - 199.1 pc
# Suggested search radius: 1 degree
coord_beehive = SkyCoord('08h40m24s', '+19d40m00s', frame='icrs')
df_beehive = queryStars(coord_beehive, 1, 162.9, 199.1)


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [4]:
clusters = {'Pleiades': df_pleiades, 
            'Hyades': df_hyades, 
            'Beehive Cluster (M44)': df_beehive}


# Function to update star positions based on proper motion and time
def update_star_positions(data, time):
    data_updated = data.copy()
    data_updated['ra'] += data['pmra'] * time / (3600 * 1000)  # Convert mas/yr to degrees
    data_updated['dec'] += data['pmdec'] * time / (3600 * 1000)  # Convert mas/yr to degrees
    return data_updated

def plot_star_motion(cluster, time):
    data = clusters[cluster]
    data_updated = update_star_positions(data, time)

    plt.figure(figsize=(8, 6))
    plt.scatter(data_updated['ra'], data_updated['dec'], marker='o', s=50)
    plt.xlabel('Right Ascension')
    plt.ylabel('Declination')
    plt.title(f'{cluster} Motion Over Time')
    plt.xlim(min(data['ra']) - 1, max(data['ra']) + 1)
    plt.ylim(min(data['dec']) - 1, max(data['dec']) + 1)
    plt.grid()
    plt.show()

cluster_dropdown = Dropdown(options=list(clusters.keys()), description='Cluster:')
time_slider = interactive(plot_star_motion, cluster=cluster_dropdown, time=(0, 10000, 10))

#display(cluster_dropdown)
display(time_slider)

interactive(children=(Dropdown(description='Cluster:', options=('Pleiades', 'Hyades', 'Beehive Cluster (M44)')…